# T5 on TPU 💥🚀

In this notebook we will see how to train T5 model on TPU with Huggingface's awesome new [trainer](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py). We will train T5 base model on SQUAD dataset for QA task. We will use the recently released amazing [nlp](https://github.com/huggingface/nlp) package to load and process the dataset in just few lines.

First make sure you are connected to the high RAM instance. This will not work on 12 GB colab instance.

In [0]:
# Crash on purpose to get more ram :
import torch
torch.tensor([10.]*10000000000)

Let's install [PyTorch/XLA](https://github.com/pytorch/xla) which enables PyTorch on TPU. Make sure you install the nightly version, as the trainer breaks on other versions.

In [1]:
VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  23821      0 --:--:-- --:--:-- --:--:-- 23821
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.7 MiB/ 91.7 MiB]                                                
Operation completed over 1 objects/91.7 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][122.1 MiB/122.1 MiB]                                                
Operation completed over 1 objects/122.1 MiB.         

Install transformers and the nlp package. Restart colab after this

In [3]:
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers


Cloning into 'transformers'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 27856 (delta 23), reused 17 (delta 6), pack-reused 27805
Receiving objects: 100% (27856/27856), 25.48 MiB | 34.33 MiB/s, done.
Resolving deltas: 100% (19331/19331), done.
Processing ./transformers
     |████████████████████████████████| 3.8MB 3.3MB/s 
     |████████████████████████████████| 1.1MB 43.7MB/s 
     |████████████████████████████████| 890kB 43.0MB/s 
  Created wheel for transformers: filename=transformers-2.10.0-cp36-none-any.whl size=665128 sha256=9fd9bd17b1f1dce1eda13eddb5e1bd652b28e749480d3fa6f3fc45d27331f939
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjm8covp/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=979919204cd7164974ee4ab3508f037752aec5e915697cb0d85abe19833e824b
  Sto

## Load and process data

Let's load and process the dataset using the nlp library. We will process the examples in follwoing way to cast QA task in text-to-text setting

**input**
question: question_text  context: context 

**target**
answer_text

In [0]:
import torch
from transformers import T5Tokenizer,T5ForConditionalGeneration

In [5]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')


In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"doanquanvietnamca","key":"5c44ad334dfc534e12d04dc8373e0440"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pip install kaggle
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

  0% 0.00/3.44M [00:00<?, ?B/s]
100% 3.44M/3.44M [00:00<00:00, 114MB/s]
 40% 5.00M/12.4M [00:00<00:00, 32.9MB/s]
100% 12.4M/12.4M [00:00<00:00, 60.6MB/s]
 99% 643M/650M [00:05<00:00, 176MB/s]
100% 650M/650M [00:05<00:00, 122MB/s]
 98% 287M/292M [00:02<00:00, 148MB/s]
100% 292M/292M [00:02<00:00, 119MB/s]
  0% 0.00/612k [00:00<?, ?B/s]
100% 612k/612k [00:00<00:00, 205MB/s]
 89% 33.0M/37.3M [00:00<00:00, 103MB/s] 
100% 37.3M/37.3M [00:00<00:00, 148MB/s]
  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 185MB/s]
 93% 74.0M/79.6M [00:00<00:00, 74.3MB/s]
100% 79.6M/79.6M [00:00<00:00, 102MB/s] 
 77% 23.0M/29.8M [00:00<00:00, 125MB/s]
100% 29.8M/29.8M [00:00<00:00, 118MB/s]


For more details on how to use the nlp library check out this [notebook](https://colab.research.google.com/github/huggingface/nlp/blob/master/notebooks/Overview.ipynb).

In [0]:
import zipfile as zf
file_name = ["jigsaw-toxic-comment-train.csv.zip", "jigsaw-unintended-bias-train.csv.zip","validation.csv.zip"]
for file in file_name:
  with  zf.ZipFile(file, 'r') as f:
    f.extractall('')

In [0]:
import pandas as pd
train1 = pd.read_csv("jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)
valid = pd.read_csv('validation.csv')
    #test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
    #sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')
train = pd.concat([
train1[['comment_text', 'toxic']],
train2[['comment_text', 'toxic']].query('toxic==1'),
train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)])
train = train.replace('\n',' ', regex=True)
valid = valid.replace('\n',' ', regex=True)

In [0]:
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer
import torch
class SegmentDataset(Dataset):
  def __init__(self, tokenizer,df,  max_len=192):
    self.data_column = df["comment_text"].values
    self.class_column = df['toxic'].values
    self.max_len = max_len
    self.tokenizer = tokenizer
        
  def __len__(self):
      return len(self.data_column)

  def __getitem__(self, index):
    input_ = '%s </s>'%(self.data_column[index])
    target = '%s </s>'%(self.class_column[index])
        # tokenize inputs
    tokenized_inputs = self.tokenizer.batch_encode_plus( [input_], max_length=self.max_len, pad_to_max_length=True,return_tensors="pt" )
    tokenized_targets = self.tokenizer.batch_encode_plus( [target], max_length=2, pad_to_max_length=True,return_tensors="pt")
    source_ids = tokenized_inputs["input_ids"].squeeze()
    target_ids = tokenized_targets["input_ids"].squeeze()
    src_mask    = tokenized_inputs["attention_mask"].squeeze() # might need to squeeze
    target_mask = torch.tensor([1,0])  # might need to squeeze
    return {"input_ids": source_ids, "attention_mask": src_mask, 
                "target_ids": target_ids, "target_attention_mask": target_mask}

In [0]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
train_dataset = SegmentDataset(tokenizer,train, 192)
valid_dataset = SegmentDataset(tokenizer, valid, 192)

In [40]:
len(train_dataset), len(valid_dataset)


(435775, 8000)

In [0]:
# cach the dataset, so we can load it directly for training

torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [0]:
del train
del valid
del train_dataset
del valid_dataset
del train1
del train2


In [0]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

# prepares lm_labels from target_ids, returns examples with keys as expected by the forward method
# this is necessacry because the trainer directly passes this dict as arguments to the model
# so make sure the keys match the parameter names of the forward method
@dataclass
class T2TDataCollator(DataCollator):
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'lm_labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=512,
        metadata={"help": "Max input length for the source text"},
    )
    target_max_len: Optional[int] = field(
        default=32,
        metadata={"help": "Max input length for the target text"},
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    #make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
        prediction_loss_only=True
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

In [0]:
import json
args_dict = {
  "num_cores": 8,
  "model_name_or_path": 't5-base',
  "max_len": 192 ,
  "target_max_len": 16,
  "output_dir": './models/tpu',
  "overwrite_output_dir": True,
  "per_device_train_batch_size": 8,
  "per_gpu_eval_batch_size": 8,
  "gradient_accumulation_steps": 4,
  "learning_rate": 1e-4,
  "tpu_num_cores": 8,
  "num_train_epochs": 4,
  "do_train": True
}
with open('args.json', 'w') as f:
  json.dump(args_dict, f)

In [45]:
!pip install -e git+https://github.com/tqdm/tqdm.git@master#egg=tqdm


Obtaining tqdm from git+https://github.com/tqdm/tqdm.git@master#egg=tqdm
  Updating ./src/tqdm clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 571f0ed3dd44aa3befc3cd326648fbb4d5b3e9a4
  Found existing installation: tqdm 4.46.0
    Can't uninstall 'tqdm'. No files were found to uninstall.
  Running setup.py develop for tqdm


In [0]:
import torch_xla.distributed.xla_multiprocessing as xmp
from tqdm import tqdm

In [0]:
xmp.spawn(_mp_fn, args=(), nprocs=8, start_method='fork')


05/28/2020 19:11:20 - INFO - transformers.training_args -   PyTorch: setting up devices
05/28/2020 19:11:20 - WARNING - __main__ -   Process rank: -1, device: xla:0, n_gpu: 0, distributed training: False, 16-bits training: False
05/28/2020 19:11:20 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./models/tpu', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=8, gradient_accumulation_steps=4, learning_rate=0.0001, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=8, tpu_metrics_debug=False)
05/28/2020 19:11:20 - INFO - transformers.training_

loading data


05/28/2020 19:11:28 - INFO - transformers.modeling_utils -   Weights of T5ForConditionalGeneration not initialized from pretrained model: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']


loading data


05/28/2020 19:11:28 - INFO - transformers.modeling_utils -   Weights of T5ForConditionalGeneration not initialized from pretrained model: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']


...

05/28/2020 21:44:38 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-3500


{"loss": 0.07423521210960461, "learning_rate": 4.8589894242068156e-05, "epoch": 2.0563959465413424, "step": 3500}


05/28/2020 21:44:38 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-3500


{"loss": 0.07659199203753815, "learning_rate": 4.8589894242068156e-05, "epoch": 2.0563959465413424, "step": 3500}


05/28/2020 21:44:38 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-3500
05/28/2020 21:44:38 - INFO - transformers.configuration_utils -   Configuration saved in ./models/tpu/checkpoint-3500/config.json
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoint-3500/pytorch_model.bin
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoint-3500/pytorch_model.bin
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoint-3500/pytorch_model.bin
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoint-3500/pytorch_model.bin
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoint-3500/pytorch_model.bin
05/28/2020 21:44:46 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/checkpoi

## Write training script

## Eval

There are two gotchas here. First the metrics functionality in the nlp package is still work-in-progress so we will use the official squad evaluation script. Second, for some reason which I couldn't figure out, the `.generate` method is not working on TPU so will need to do prediction on CPU. For predicting the validation set it almost takes 40 mins.

In [0]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

import nlp
from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import tqdm

In [0]:
model = T5ForConditionalGeneration.from_pretrained('models/tpu').to('cpu') # because its loaded on xla by default
tokenizer = T5Tokenizer.from_pretrained('models/tpu')

In [0]:
valid_dataset = torch.load('valid_data.pt')
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)

In [0]:
answers = []
for batch in tqdm(dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=16,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

In [0]:
predictions = []
references = []
for ref, pred in zip(valid_dataset, answers):
  predictions.append(pred)
  references.append(ref['answers']['text'])

In [0]:
predictions[0], references[0]

('Denver Broncos', ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'])

In [0]:
evaluate(references, predictions)

{'exact_match': 81.56102175969725, 'f1': 89.96016967193422}